In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import altair as alt
alt.renderers.enable('mimetype')
alt.data_transformers.disable_max_rows()
import networkx as nx
import os

In [14]:
# user_df = pd.read_csv("../data/entity_files/users_dataset.csv")
# repo_df = pd.read_csv("../data/large_files/entity_files/repos_dataset.csv", low_memory=False)
# org_df = pd.read_csv("../data/entity_files/orgs_dataset.csv", low_memory=False)
initial_core_users = pd.read_csv("../data/derived_files/initial_core_users.csv")
initial_core_users['origin'] = 'initial_core'
initial_core_repos = pd.read_csv("../data/derived_files/initial_core_repos.csv")
initial_core_repos['origin'] = 'initial_core'
initial_core_orgs = pd.read_csv("../data/derived_files/initial_core_orgs.csv")
initial_core_orgs['origin'] = 'initial_core'

firstpass_core_users = pd.read_csv("../data/derived_files/firstpass_core_users.csv")
firstpass_core_users['origin'] = 'firstpass_core'
firstpass_core_repos = pd.read_csv("../data/derived_files/firstpass_core_repos.csv")
firstpass_core_repos['origin'] = 'firstpass_core'
firstpass_core_orgs = pd.read_csv("../data/derived_files/firstpass_core_orgs.csv")
firstpass_core_orgs['origin'] = 'firstpass_core'

finalpass_core_users = pd.read_csv("../data/derived_files/finalpass_core_users.csv")
finalpass_core_users['origin'] = 'finalpass_core'
finalpass_core_repos = pd.read_csv("../data/large_files/derived_files/finalpass_core_repos.csv", low_memory=False, on_bad_lines='skip')
finalpass_core_repos['origin'] = 'finalpass_core'
finalpass_core_orgs = pd.read_csv("../data/derived_files/finalpass_core_orgs.csv")
finalpass_core_orgs['origin'] = 'finalpass_core'

In [15]:
core_users = pd.concat([initial_core_users, firstpass_core_users, finalpass_core_users])
core_repos = pd.concat([initial_core_repos, firstpass_core_repos, finalpass_core_repos])
core_orgs = pd.concat([initial_core_orgs, firstpass_core_orgs, finalpass_core_orgs])

- user-user interactions:
  - org_members
  - user_orgs
  - user_followers
  - user_following

- user-repo interactions:
  - user_repos
  - contirbutors_repos
  - starrers_repos
  - subscribers_repos
  - forkers_repos
  - committers_repos
  - 

In [68]:
dfs = []
for dir, subdir, files in os.walk("../data/join_files/"):
    for f in files:
        if ('search' not in f) and (f.endswith(".csv")):
            entity_type = f.split("_")[0]
            print(f)
            df = pd.read_csv(os.path.join(dir, f), nrows=10)
            cols = df.columns
            cols = [col for col in cols if any(x in col for x in ['full_name', 'login'])]
            cols = [col for col in cols if ('.' not in col) or ('user.login' in col)]
            if len(cols) > 2:
                cols = ['user.login', 'repo_full_name']
            if len(cols) == 1:
                cols = cols + ['author.login']
            df_dict = {}
            df_dict['entity_type'] = entity_type
            
            df_dict['source'] = cols[0] if 'forks' not in f else "owner.login"
            df_dict['target'] = cols[1]
            df_dict['file_name'] = os.path.join(dir, f)
            dfs.append(df_dict)

for dir, subdir, files in os.walk("../data/large_files/join_files/"):
    for f in files:
        if ('search' not in f) and (f.endswith(".csv")) and ('commits' not in f):
            entity_type = f.split("_")[0]
            print(f)
            df = pd.read_csv(os.path.join(dir, f), nrows=10)
            cols = df.columns
            cols = [col for col in cols if any(x in col for x in ['full_name', 'login'])]
            cols = [col for col in cols if ('.' not in col) or ('user.login' in col)]
            if len(cols) > 2:
                cols = ['user.login', 'repo_full_name']
            if len(cols) == 1:
                cols = cols + ['author.login']
            df_dict = {}
            df_dict['entity_type'] = entity_type
            df_dict['source'] = cols[0] if 'forks' not in f else "owner.login"
            df_dict['target'] = cols[1]
            df_dict['file_name'] = os.path.join(dir, f)
            dfs.append(df_dict)

org_members_join_dataset.csv
org_followers_join_dataset.csv
repo_subscribers_join_dataset.csv
repo_orgs_join_dataset.csv
org_repos_join_dataset.csv
user_repos_join_dataset.csv
user_following_join_dataset.csv
repo_comments_join_dataset.csv
user_subscriptions_join_dataset.csv
issues_comments_join_dataset.csv
repo_subscribers_join_dataset.csv
user_followers_join_dataset.csv
user_starred_join_dataset.csv
repo_stargazers_join_dataset.csv
repo_contributors_join_dataset.csv
repo_issues_join_dataset.csv
repo_pulls_join_dataset.csv
user_orgs_join_dataset.csv
pulls_comments_join_dataset.csv
repo_forks_join_dataset.csv


In [69]:
cols_df = pd.DataFrame(dfs)

We will initially have bipartite networks but I'm interested in projecting them to find communities as well

In [72]:
cols_df['network_type'] = None
for index, row in cols_df.iterrows():
    if row.target == 'org_login':
        network_first = "org"
        network_second = 'user' if row.source == 'login' else 'repo'
        network_type = network_first + "_" + network_second
        cols_df.loc[index, 'network_type'] = network_type
    if row.target == 'repo_full_name':
        network_first = "repo"
        if 'orgs' in row.file_name:
            network_second = 'org'
        else:
            network_second = 'user' if 'login' in row.source else 'repo'
        network_type = network_first + "_" + network_second
        cols_df.loc[index, 'network_type'] = network_type
    if row.target == 'user_login':
        network_first = "user"
        if "orgs" in row.file_name:
            network_second = 'org'
        else:
            network_second = 'user' if 'login' in row.source else 'repo'
        network_type = network_first + "_" + network_second
        cols_df.loc[index, 'network_type'] = network_type
    if row.source == 'repo_full_name':
        network_type = "repo_user"
        cols_df.loc[index, 'network_type'] = network_type


In [75]:
cols_df.groupby('network_type').size()

network_type
org_repo      1
org_user      2
repo_org      1
repo_user    10
user_org      1
user_repo     3
user_user     2
dtype: int64

In [54]:
# def build_networks(cols_df):
orgs_dfs =[]
users_dfs = []
repos_dfs = []

for index, row in cols_df[(cols_df.target == 'org_login') & (cols_df.source == 'login')].iterrows():
    df = pd.read_csv(row['file_name'], low_memory=False)
    target_type = row.target
    source_type = row.source
    print(target_type, source_type, row.file_name)
    if 'org' in target_type:
        filtered_df = df[(df[row.target].isin(core_orgs[core_orgs.origin == 'initial_core'].login))]
        grouped_filtered_df = filtered_df.groupby([target_type, source_type]).size().reset_index(name='counts')
        grouped_filtered_df['entity_type'] = row['entity_type']
        grouped_filtered_df['file_path'] = row['file_name']
        grouped_filtered_df['join_type'] ='_'.join(row['file_name'].split('/')[-1].split('_')[0:2])
        orgs_dfs.append(grouped_filtered_df)


org_login login ../data/join_files/org_members_join_dataset.csv
org_login login ../data/join_files/org_followers_join_dataset.csv


In [55]:
combined_orgs_dfs = pd.concat(orgs_dfs)

In [56]:
combined_orgs_dfs[0:1].to_dict()

{'org_login': {0: 'ABC-DH'},
 'login': {0: 'digitalkoine'},
 'counts': {0: 1},
 'entity_type': {0: 'org'},
 'file_path': {0: '../data/join_files/org_members_join_dataset.csv'},
 'join_type': {0: 'org_members'}}

In [57]:
bipartite_graph = nx.Graph()

In [42]:
user_nodes = core_users['login'].tolist()
repo_nodes = core_repos['full_name'].tolist()
org_nodes = core_orgs['login'].tolist()
user_nodes = user_nodes + org_nodes

In [32]:
len(user_nodes), len(repo_nodes), len(edges)

(793, 2264, 4983)

In [43]:
bipartite_graph.add_nodes_from(user_nodes, bipartite=0)
bipartite_graph.add_nodes_from(repo_nodes, bipartite=1)
bipartite_graph.add_edges_from(edges)

In [44]:
print('connected?', nx.is_connected(bipartite_graph))
print('bipartite?', nx.is_bipartite(bipartite_graph))

connected? False
bipartite? True


In [45]:
nx.write_gexf(bipartite_graph,"../data/derived_files/bipartite_graph.gexf")

In [2]:
test = pd.read_excel("../data/private_data/Appendix A publication_raw_chun_llc_dhq.xlsx")

In [9]:
test2 = pd.read_excel("../data/private_data/Appendix E co-retweet network.xlsx")

In [10]:
test2

,Unnamed: 0,label,handle,no. tweet,no. retweet received,no. nonself retweet received,retweeted %,betweenness centrality,gender,country,most retweeting hashtag,most retweeted hashtag,account create year
0,1,ADHO,ADHOrg,2595,3028,2590,0.998073,179742.715164,U,Unknown,digitalhumanities(114); dh2014(87); dh2015(83)...,digitalhumanities(665); dh2016(406); dh2017(27...,2013
1,966,NEH,NEHgov,3232,4093,4145,1.282488,103.811686,U,United States,humanities(77); nehgrant(54); cubavisit(32); n...,nehgrant(504); neh(271); humanities(196); otd(...,2010
2,340,melissa terras,melissaterras,3202,2219,3899,1.217676,589.969181,F,United Kingdom,digitalhumanities(16); dh2017(15); bldigital(6...,digitalhumanities(548); dh2017(158); dh2014(12...,2009
3,4,aaDH,aaDHumanities,1199,250,158,0.131776,121131.629443,U,Australia,digitalhumanities(92); dha2016(66); humanities...,dha2016(83); digitalhumanities(79); eresearch(...,2015
4,223,Bethany Nowviskie,nowviskie,3222,2463,4366,1.355059,1266.470119,F,United States,dlfforum(233); endangereddata(64); dlflac(54);...,dlfforum(357); dh2013(113); endangereddata(109...,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3146,3147,yhkondo,yhkondo,830,0,8,0.009639,0.000000,M,Japan,androidapp(1); asteriskdb0090(1); epub_tw(1); ...,0,2009
3147,3148,Yuji Shinozaki,ys2n,284,0,0,0.000000,0.000000,M,Unknown,0,0,2008
3148,3149,Yutzu,yutzudotcom,136,6,8,0.058824,0.000000,F,Canada,0,brazil(1); curator(1); historian(1); librarian...,2010
3149,3150,Zeus Oratrios,Zeus_Kairos,1923,0,0,0.000000,0.000000,M,Unknown,notav(11); venezia(9); art(8); fb(6); terremot...,0,2009


In [8]:
test

,Authors,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,Page count,...,Conference code,ISSN,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Source,EID
0,"Spencer M., Wachtel K., Howe C.J.",Representing multiple pathways of textual flow...,2004,Computers and the Humanities,38,1,NaN,1,14,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,-,Article,Scopus,2-s2.0-26844469206
1,"Collins J., Kaufer D., Vlachos P., Butler B., ...",Detecting collaborations in text,2004,Computers and the Humanities,38,1,NaN,15,36,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,-,Article,Scopus,2-s2.0-28344440777
2,"Can F., Patton J.M.",Change of writing style with time,2004,Computers and the Humanities,38,1,NaN,61,82,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,-,Article,Scopus,2-s2.0-33845685623
3,"Pohl M., Purgathofer P.",Hypertext writing profiles and visualisation,2004,Computers and the Humanities,38,1,NaN,83,105,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,-,Article,Scopus,2-s2.0-33845692183
4,Wilks Y.,On the ownership of text,2004,Computers and the Humanities,38,2,NaN,115,127,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,-,Article,Scopus,2-s2.0-33845693781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,[No author name available],The Thomas Mann project: A progress report,1966,Computers and the Humanities,1,3,NaN,65,71,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,Comput Hum,Article,Scopus,2-s2.0-34250504503
1031,[No author name available],The computer and Plato's Seventh letter,1966,Computers and the Humanities,1,3,NaN,72,73,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,Comput Hum,Article,Scopus,2-s2.0-34250519129
1032,[No author name available],The new center for research at the University ...,1966,Computers and the Humanities,1,3,NaN,73,74,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,Comput Hum,Article,Scopus,2-s2.0-34250514213
1033,[No author name available],Literary works in machine-readable form,1966,Computers and the Humanities,1,3,NaN,75,102,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,Comput Hum,Article,Scopus,2-s2.0-34250541874
